<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Import required libraries





In [5]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!pip install geocoder
#import geocoder

!pip install geopy
import geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library





### Download data and place in data frame

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#
postalTables = pd.read_html(url,match="Postal Code")
#postalTables[0]
postalTablesDataFrame = pd.DataFrame(postalTables[0])
postalTablesDataFrame.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove Neibourhods without boroughs

<b>Assumption</b>: All neighbourhoods with state <b>Not assigned</b> also have boroughs with state  <b>Not assigned</b>. 

In [7]:
postalTables_wiithBoroughs = postalTablesDataFrame[postalTablesDataFrame.Borough != 'Not assigned']
postalTables_wiithBoroughs.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Check if any neigbourhood exist that are not assigned 

In [8]:
No_neighbourhoodsAssigned = postalTables_wiithBoroughs.loc[postalTables_wiithBoroughs['Neighbourhood'] =='Not assigned'] 
No_neighbourhoodsAssigned

,Postal Code,Borough,Neighbourhood


In [9]:
postalTables_wiithBoroughs.shape

(103, 3)

<b>Google Maps Geocoding API package was not working so I will be using csv to get longditude and lattitude.</b>

In [10]:
latlong = pd.read_csv('https://cocl.us/Geospatial_data')  
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
#neighbourhoodsAssignedLL =pd.merge(postalTables_wiithBoroughs,latlong,how='left',left_on='Postal Code',right_on='Postal Code').drop(['Postal Code'], axis=1)
neighbourhoodsAssignedLL =pd.merge(postalTables_wiithBoroughs,latlong,how='left',left_on='Postal Code',right_on='Postal Code')
neighbourhoodsAssignedLL.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
neighbourhoodsAssignedLL.shape

(103, 5)

Use geopy library to get the latitude and longitude values of Toronto.

In [13]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


#### Create map with neihbourhoods

In [20]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighbourhoodsAssignedLL['Latitude'], neighbourhoodsAssignedLL['Longitude'], neighbourhoodsAssignedLL['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#4186fc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

